# Cart Pendulum dynamics: Lagrangian mechanics

In [1]:
from ipywidgets import interact
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

import sympy  as sp
import numpy as np
import rich

%matplotlib inline

# Deriving the equations
We will derive the equations using Euler–Lagrange equation.

The Pendulum is assumed to be a mass at the end of a massless rod

The Variables 

x : Position of the cart

theta : Angles of the pendulum measured from the y axis

#### Kinematics

In [ ]:
# Time
t  = sp.symbols('t')

# Time derivative
ddt = lambda x:sp.diff(x,t)

# States
x   = sp.Function('x')
theta   = sp.Function('theta')
xd  = ddt(x(t))
d_theta  = ddt(theta(t))
xdd = ddt(xd)
d2_theta = ddt(d_theta)

# Control input variable
u   = sp.symbols('u')

# Parameters
m_c, m_p, l, g = sp.symbols('m_c, m_p, l, g')

# positions. Here c: cart, p: pendulum
cart_x   = x(t)
cart_y   = 0
cart_pos = [cart_x, cart_y]

pendulum_x   =  l*sp.sin(theta(t)) + x(t)
pendulum_y   =  l*sp.cos(theta(t))
pendulum_pos = [pendulum_x, pendulum_y]

# Velocities
cart_vel     = [ddt(i) for i in cart_pos]
pendulum_vel = [ddt(i) for i in pendulum_pos]

cart_v_squared     = sum(i**2 for i in cart_vel)
pendulum_v_squared = sum(i**2 for i in pendulum_vel)

#### Solving the Langrange equations

In [ ]:
# Energies
# Potential energy
pe = -m_p*g*pendulum_y

# Kinetic energy
ke = m_c*cart_v_squared/2   +   m_p*pendulum_v_squared/2

# Lagrangian
L = ke - pe

# The Differential equation
DEs = [
    ddt(sp.diff(L, xd)) - sp.diff(L, x(t)) - u,
    ddt(sp.diff(L, d_theta)) - sp.diff(L, theta(t))
]
DEs = [DE.simplify() for DE in DEs]

# Solving to get explicit values for the two state double derivatives
DEexplicit = sp.solve(DEs,           [xdd, d2_theta]) # Solving
DEexplicit = [DEexplicit[i] for i in [xdd, d2_theta]] # Extracting the solutions
DEexplicit = [DE.simplify() for DE in DEexplicit]  # Simplifying the solution

#### Verification test: Center of mass motion

In [ ]:
x_cm = (m_p*pendulum_x + m_c*cart_x )/(m_c + m_p)
ddt(ddt(x_cm.simplify())).simplify()

#### Expressions to functions

In [ ]:
y1, y2, y1d, y2d = sp.symbols('y1, y2, y1d, y2d')
substitutions = {
    x(t): y1,
    theta(t): y2,

    ddt(x(t)): y1d,
    ddt(theta(t)): y2d,
}

y1dd, y2dd    = [i.subs(substitutions) for i in DEexplicit]
y1dd_function = sp.lambdify((y1, y1d, y2, y2d, t, u, m_c, m_p, l, g), y1dd) 
y2dd_function = sp.lambdify((y1, y1d, y2, y2d, t, u, m_c, m_p, l, g), y2dd) 

# Simulating 

In [ ]:
def get_PendulumCartRates(X, t, uf, m_c, m_p, l, g):
    X_ = X
    u  = uf(X_)
    x, v1, theta, v2 = X_
    xd = v1
    v1d = y1dd_function(x, v1, theta, v2, t, u, m_c, m_p, l, g)
    d_theta = v2
    v2d = y2dd_function(x, v1, theta, v2, t, u, m_c, m_p, l, g)


    return np.array([xd, v1d, d_theta, v2d])

In [ ]:
# System parameters
param_vals = {
    "m_c" : 1,
    "m_p" : 1,
    "g"  : -10,
    "l"  : 1
}

# No control law, free moving
control_no = lambda x:0 

# Simulation parameters
dt = 0.001
t0 = 0.0
tn = 10.0
ts = np.arange(t0, tn, dt)
t_step = [0.0, dt]

# Initial condition
x0 = np.array([0, 0, np.pi/2, 0])

# Integrate
result = odeint(
    get_PendulumCartRates, x0, ts, 
    args=(control_no, param_vals["m_c"], param_vals["m_p"], param_vals["l"], param_vals["g"])
    )

### Plotting setting
Makes the plots beautiful

In [ ]:
# plt.style.use('bmh')
# plt.style.use('ggplot')
# plt.style.use('classic')
plt.style.use('fivethirtyeight')

plt.rcParams["figure.figsize"]   = (6,6)
plt.rcParams["figure.dpi"]       = 100
plt.rcParams["figure.facecolor"] = "ffffff"

plt.rcParams["savefig.pad_inches"] = 0.1
plt.rcParams["savefig.bbox"]       = "tight"
plt.rcParams["savefig.facecolor"]  = "ffffff"
plt.rcParams["savefig.dpi"]        = 100

# General font
plt.rcParams["font.size"]        = 14
plt.rcParams["text.color"]       = "black"

# Axis text properties
plt.rcParams["axes.labelsize"]   = 20
plt.rcParams["axes.labelweight"] = 1000
plt.rcParams["xtick.color"]      = "black"
plt.rcParams["ytick.color"]      = "black"

# The graph outline, background and grid properties
plt.rcParams["axes.facecolor"]   = "d6ebf2" # "e0e0e0"
plt.rcParams["axes.grid"]        = True          
plt.rcParams["grid.color"]       = "white"
plt.rcParams["axes.labelcolor"]  = "black"
plt.rcParams["axes.edgecolor"]   = "57adc7" # "888888" # "black"
plt.rcParams["axes.linewidth"]   = 1

# Padding between border and text
plt.rcParams["axes.titlesize"]   = 25
plt.rcParams["axes.titlepad"]    = 10
plt.rcParams["axes.titleweight"] = 500

# Legend position reference point
plt.rcParams["legend.loc"]       = "lower center"
plt.rcParams["legend.frameon"]   = True
# plt.rcParams["legend.shadow"]    = True
# plt.rcParams["legend.facecolor"] = "e0e0e0"

# The line plot thickness and colors
plt.rcParams["lines.linewidth"]  = 2.5
# plt.rcParams["axes.prop_cycle"]  = plt.cycler('color', [
#       "#eb7245",  "#0066a2",  "#f1be3e",  "#007188",  "#82d7c6" , "#82d7c6", "#c3312f"
#     ])



In [ ]:
def animte_CartPend(result, ts, x_range = [-3,   3], y_range = [-1.1, 1.1], scale = 3):
    """
    Accepts cart state information and returns an animation
    """
    # Setting axes range
    delta   = lambda ls: ls[1] - ls[0]
    aspect  = (delta(x_range)) / (delta(y_range))

    # setting matplotlib parameters
    plt.rcParams['figure.figsize'] = [aspect * scale, scale]
    plt.rcParams.update({'font.size': 18})
    plt.rcParams['animation.html'] = 'jshtml'


    # Selecting the animation range
    res_plot = result[::50]
    ts_plot  = ts[::50]

    # Setting up the plot
    fig, ax     = plt.subplots()
    pend_plot,  = plt.plot([],[], 'o-', linewidth=2, ms=10, markerfacecolor='r')
    cart_plot,  = plt.plot([],[], 'ks', ms=10) 

    print(type(pend_plot))

    def init():
        ax.set_xlim(x_range[0], x_range[1])
        ax.set_ylim(y_range[0], y_range[1])
        ax.set_aspect('equal')

    def animate(iter):
        x_cart, _, th_pend, _ = res_plot[iter]
        sub_obs               = {x(t): x_cart, theta(t): th_pend, l: param_vals["l"]}
        x_pend, y_pend        = [sp.N(i.subs(sub_obs)) for i in pendulum_pos]

        pend_plot.set_data([x_cart, x_pend],[0, y_pend])
        cart_plot.set_data([x_cart]        ,[0])

    return animation.FuncAnimation(fig, animate, init_func=init, frames=len(res_plot), interval=50, blit=False, repeat=False)

In [ ]:
anim = animte_CartPend(result, ts, x_range=[-1,2], scale=5)
HTML(anim.to_jshtml())